In [1]:
from data_loader import load_pathway_data
from data_loader import load_gene_families
from data_loader import intersect
from data_loader import union
from data_loader import align_tensor
from scipy.io import mmread
import pandas as pd
import numpy as np

In [ ]:
# Convert sparse matrix to DataFrame and save as CSV
matrix = mmread("/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012/raw/gene_abundance_raw/gene_abundance.mtx").toarray()
cols_names_file = "/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012/raw/gene_abundance_colnames.txt"
rows_names_file = "/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012/raw/gene_abundance_rownames.txt"
output_path = "/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012/raw/gene_abundance_raw/gene_abundance.csv"
row_names = []
col_names = []

# Read row and column names
with open(rows_names_file, "r") as f:
    row_names = [line.strip() for line in f]

with open(cols_names_file, "r") as f:
    col_names=[line.strip() for line in f]
    
# Check dimensions for safety
if matrix.shape[0] != len(row_names):
    raise ValueError(f"Number of rows in matrix ({matrix.shape[0]}) does not match number of row names ({len(row_names)})")

if matrix.shape[1] != len(col_names):
    raise ValueError(f"Number of columns in matrix ({matrix.shape[1]}) does not match number of column names ({len(col_names)})")

# Convert to DataFrame
df = pd.DataFrame(matrix, index=row_names, columns=col_names)

# Save as CSV
df.to_csv(output_path)

In [ ]:
def filter_stool_samples(metadata_path, df_path, output_path):
    """
    Filters abundance table to include only stool samples.
    """
    # Load metadata and get stool sample IDs
    metadata = pd.read_csv(metadata_path)
    metadata = metadata[metadata['body_site'] == 'stool']
    stool_samples = metadata.iloc[:, 0].tolist()

    # Load full gene abundance data
    df = pd.read_csv(df_path, index_col=0)

    # Filter columns (samples) to include only stool samples
    df = df[stool_samples]

    # Save to csv
    df.to_csv(output_path)

In [ ]:
filter_stool_samples(
    metadata_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012/raw/gene_abundance_col_metadata.csv",
    df_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/raw/gene_abundance.csv",
    output_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/raw/gene_abundance_stool.csv"
)

filter_stool_samples(
    metadata_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012/raw/gene_abundance_col_metadata.csv",
    df_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/raw/pathway_abundance.csv",
    output_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/raw/pathway_abundance_stool.csv"
)

In [7]:
import pandas as pd

def filter_uniref_to_bacteria(rel_abundance_path, uniref_path, output_path, gene_abundance=True):
    """
    Filters the UniRef file to retain only rows corresponding to bacterial taxa,
    based on a relative abundance file where taxa names are in the columns.

    Parameters:
    - rel_abundance_path: str – path to relative abundance file (taxa in columns, samples in rows)
    - uniref_path: str – path to UniRef file (taxa in index, samples in columns)
    - output_path: str – path to save filtered file containing only bacterial taxa
    """

    rel_abundance = pd.read_csv(rel_abundance_path, index_col=0)

    if gene_abundance: # gene abundance, tsv format
        uniref_abundance = pd.read_csv(uniref_path, index_col=0, sep='\t')
    else: # pathway abundance, csv format
        uniref_abundance = pd.read_csv(uniref_path, index_col=0)

    bacteria_taxa = [col for col in rel_abundance.columns if col.startswith("k__Bacteria")]

    def extract_genus_species(taxon):
        parts = taxon.split('|')
        genus = next((p for p in parts if p.startswith("g__")), "")
        species = next((p for p in parts if p.startswith("s__")), "")
        if genus and species:
            return f"{genus}.{species}"
        return None

    valid_taxa = set(filter(None, [extract_genus_species(t) for t in bacteria_taxa]))
    uniref_index_taxa = uniref_abundance.index.to_series().str.extract(r'\|(g__.*?\.s__.*)')[0].fillna("")
    is_bacteria = uniref_index_taxa.isin(valid_taxa)
    uniref_bacteria = uniref_abundance[is_bacteria]
    uniref_bacteria.to_csv(output_path)

In [10]:

def expand_taxonomies(npy_path, rel_abundance_path, output_path=None):
    # Load the short-format taxonomy names (g__...s__...) from .npy
    short_taxa = np.load(npy_path, allow_pickle=True)

    # Load the relative abundance file (columns are full taxonomies)
    rel_abundance = pd.read_csv(rel_abundance_path, index_col=0)
    full_taxa = rel_abundance.columns.tolist()

    # Build a lookup dictionary: g__...s__... → full_taxonomy
    mapping = {}
    for full in full_taxa:
        parts = full.split('|')
        genus = next((p for p in parts if p.startswith('g__')), '')
        species = next((p for p in parts if p.startswith('s__')), '')
        if genus and species:
            key = f"{genus}.{species}"
            mapping[key] = full

    # Map the short names to full taxonomy
    expanded = []
    for name in short_taxa:
        full_name = mapping.get(name, None)
        if full_name:
            expanded.append(full_name)
        else:
            print(f"⚠️ No match found for: {name}")
            expanded.append(name)  # optionally leave as-is or use None

    expanded = np.array(expanded)

    # Save if output path provided
    if output_path:
        np.save(output_path, expanded)
        print(f"✅ Saved expanded taxonomy to {output_path}")
    
    return expanded


In [12]:
expanded_taxa = expand_taxonomies(
    npy_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/test_bacteria.npy",
    rel_abundance_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012/raw/relative_abundance.csv",
    output_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/bacteria_names_full_taxonomy.npy"
)

✅ Saved expanded taxonomy to /home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/bacteria_names_full_taxonomy.npy


In [8]:
filter_uniref_to_bacteria(
    rel_abundance_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012/raw/relative_abundance.csv",
    uniref_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/raw/gene_abundance_stool_regroup.tsv",
    output_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/raw/gene_abundance_filter.csv")

filter_uniref_to_bacteria(
    rel_abundance_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012/raw/relative_abundance.csv",
    uniref_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/raw/pathway_abundance_stool.csv",
    output_path="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/raw/pathway_abundance_filter.csv",
    gene_abundance=False)  # Set to False for pathway abundance

In [ ]:
# Stage 1: Process geneabundance data
# Remove unclassified microbs and unmapped reads

input_path = "/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/raw/gene_abundance_filter.csv"
output_dir = "/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/genefamilies/before_intersection"

load_gene_families(input_path, output_dir, threshold=False, top_k=1000)

Processing rows: 100%|██████████| 189063/189063 [00:08<00:00, 23009.01it/s]


Tensor shape before normalization: torch.Size([147, 600, 2205])
Tensor shape after normalization: torch.Size([147, 600, 2205])
Sample sums after normalization (should all be 1.0): tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000], dtype=torch.float64)


In [6]:
# write to a file the bacteria names
names = np.load("../data/HMP_2012/genefamilies/before_intersection/bacteria_list.npy", allow_pickle=True)
with open("../data/HMP_2012/genefamilies/bacteria_list.txt", "w") as f:
    for name in names:
        f.write(f"{name}\n")


In [ ]:
# Stage 2: load appropirate pathways.

input_path = "/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/raw/pathway_abundance_filter.csv"
output_dir = "/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/pathabundance/before_intersection"

load_pathway_data(input_path, output_dir)

torch.Size([147, 540, 380])


In [5]:
# Sanity check for the tensor. 
bacteria_list = np.load("../data/HMP_2012/pathabundance/before_intersection/bacteria_list.npy", allow_pickle=True)
people_list = np.load("../data/HMP_2012/pathabundance/before_intersection/sample_list.npy", allow_pickle=True)
pathway_list = np.load("../data/HMP_2012/pathabundance/before_intersection/pathway_list.npy", allow_pickle=True)
tensor = np.load("../data/HMP_2012/pathabundance/before_intersection/tensor.npy")

# find the indices for the sanity check
t1 = list(people_list).index('SRS011061')
t2 = list(bacteria_list).index('g__Bacteroides.s__Bacteroides_cellulosilyticus')
t3 = list(pathway_list).index('DTDPRHAMSYN-PWY: dTDP-L-rhamnose biosynthesis I')
"""
t1 = np.where(people_list == 'ERR9830187')[0][0]
t2 = np.where(bacteria_list == 'g__Escherichia.s__Escherichia_coli')[0][0]
t3 = np.where(gene_family_list == 'GO:0000006')[0][0]
"""
print(f"the value for the sanity check is: {tensor[t1][t2][t3]}")

the value for the sanity check is: 0.000110522


In [ ]:
# Stage 3 (optional): Given n tensors from the shape: [samples, bacteria, pathways] perform an union.

input_path = ["data/data_files/pathways/AsnicarF_2017_march", "data/data_files/pathways/AsnicarF_2021_march"]
output_dir = "data/data_files/pathways/Union/"
union(input_path, output_dir, is_pathway=True)

In [ ]:
# Stage 4 (optional): Given n tensors from the shape: [samples, bacteria, gene_families] perform an union.

input_path = ["data/data_files/gene_families/AsnicarF_2017_march", "data/data_files/gene_families/AsnicarF_2021_march"]
output_dir = "data/data_files/gene_families/Union/"
union(input_path, output_dir, is_pathway=False)

In [9]:
# Stage 5: intersect between pathway abundances and gene families through the samples and bacteria dimensions."

raw_gene_families= "/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool//genefamilies/before_intersection"
raw_pathways= "/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/pathabundance/before_intersection"
intersected_gene_families="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/genefamilies/after_intersection"
intersected_pathways="/home/barsapi1/metric/Bacteria-Metric/data/HMP_2012_stool/pathabundance/after_intersection"
intersect(raw_gene_families, raw_pathways, intersected_gene_families, intersected_pathways)

Intersected gene families tensor shape: (147, 540, 2205)
Complementary gene families tensor shape: (147, 60, 2205)
Intersected pathways tensor shape: (147, 540, 380)
